# 00: Extracción de Legislaturas y Períodos

**Propósito:** Este *notebook* extrae la lista completa de **legislaturas** desde la API WSDL de la Cámara. El *output* de la API es una lista denormalizada donde cada fila es una legislatura y contiene la información del período al que pertenece.

**Salidas (Artifacts):**
1.  `data/01_raw/legislaturas_completas.csv`: Un archivo crudo (sin limpiar) con todas las legislaturas.
2.  `data/01_raw/periodos_master.csv`: Una **lista maestra de períodos** únicos (derivada de la anterior), que servirá como *input* para los *notebooks* `01_Extraction_Deputies` y `03_Extraction_Votes`.
3.  Archivos de detalle (`legislaturas.csv`) guardados por cada período.

In [1]:
import pandas as pd
from pathlib import Path
from zeep import Client
import sys
import logging

# --- Configurar Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Importar lógica personalizada de /src ---
sys.path.append('../') 
try:
    # Asumimos que su función 'get_legislaturas' vive en 'src/extraction_utils.py'
    from src.extraction_utils import get_legislaturas
    from src.common_utils import safe_serialize
except ImportError:
    logging.error("Error: No se pudieron importar las funciones desde 'src/extraction_utils.py'.")
    raise

In [ ]:
# --- 1. Configuración de Rutas y Constantes ---

WSDL_URL = "https://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx?WSDL"
ROOT = Path.cwd().parent
DATA_DIR_RAW = ROOT / "data" / "01_raw"

DATA_DIR_RAW.mkdir(parents=True, exist_ok=True)

logging.info(f"Ruta Raíz: {ROOT}")
logging.info(f"Directorio de Datos Raw: {DATA_DIR_RAW}")

## 2. Extracción de Legislaturas (Datos Crudos)

Se conecta a la API y se descarga la lista denormalizada de legislaturas. Este DataFrame se guarda **tal cual (crudo)** para asegurar la trazabilidad.

In [ ]:
try:
    client = Client(WSDL_URL)
    logging.info(f"Cliente WSDL inicializado desde: {WSDL_URL}")

    # 1. Llamar a la función de lógica
    # df_legislaturas es un DataFrame de LEGISLATURAS (con datos de período repetidos)
    df_legislaturas = get_legislaturas(client) 
    
    # 2. Guardar el archivo maestro crudo (SIN LIMPIAR NOMBRES)
    RAW_LEGISLATURAS_FILE = DATA_DIR_RAW / "legislaturas_completas.csv"
    df_legislaturas.to_csv(RAW_LEGISLATURAS_FILE, index=False, encoding="utf-8")
    
    logging.info(f"Se extrajeron {len(df_legislaturas)} legislaturas.")
    logging.info(f"Archivo crudo guardado en: {RAW_LEGISLATURAS_FILE}")
    
    # display(df_legislaturas.head())

except Exception as e:
    logging.error(f"ERROR FATAL al extraer legislaturas: {e}")
    raise

## 3. Creación de la Lista Maestra de Períodos

A partir del *dataframe* de legislaturas, se extraen las columnas que definen un **Período** y se eliminan duplicados.

Este `periodos_master.csv` es el archivo que consumirán los siguientes *notebooks* de extracción.

In [ ]:
# 1. Crear el DataFrame 'df_periodos' de forma inequívoca.
# Seleccionamos las primeras 4 columnas por su POSICIÓN (iloc)
# y las renombramos canónicamente en el acto.
try:
    df_periodos = pd.DataFrame({
        "Id": df_legislaturas.iloc[:, 0],
        "Nombre": df_legislaturas.iloc[:, 1],
        "FechaInicio": df_legislaturas.iloc[:, 2],
        "FechaTermino": df_legislaturas.iloc[:, 3],
    })
except IndexError:
    logging.error("ERROR: df_legislaturas no tiene las 4 columnas esperadas en las posiciones 0-3.")
    raise

# 2. Ahora 'df_periodos' es un DataFrame limpio y sin ambigüedades.
# El resto de la lógica funciona perfectamente.
df_periodos = (
    df_periodos
    .drop_duplicates(subset=["Id"]) # Usar 'Id' (ID de Período) como clave única
    .sort_values(by="FechaInicio")
    .reset_index(drop=True)
)

# 3. Guardar la lista maestra
MASTER_PERIOD_FILE = DATA_DIR_RAW / "periodos_master.csv"
df_periodos.to_csv(MASTER_PERIOD_FILE, index=False, encoding="utf-8")

logging.info(f"Lista maestra de {len(df_periodos)} períodos guardada en: {MASTER_PERIOD_FILE}")
display(df_periodos.head())

## 4. Guardado de Artefactos por Período

Finalmente, se itera sobre la lista de períodos únicos y se guarda un archivo `legislaturas.csv` para cada uno, facilitando la inspección de los datos.

In [ ]:
logging.info("Iniciando guardado de archivos de legislatura por período...")

# Usar 'itertuples()'
for row in df_periodos.itertuples():
    
    # 'row.Nombre' viene de 'df_periodos', que SÍ es limpio
    periodo_nombre = row.Nombre 
    
    try:
        # 1. Filtramos 'df_legislaturas' usando la columna de 'Nombre'
        #    del período, que sabemos está en la POSICIÓN 1 (iloc[:, 1]).
        
        # Seleccionamos la columna de nombre de período por su índice
        columna_nombre_periodo = df_legislaturas.iloc[:, 1]
        
        # Filtramos el DataFrame crudo usando esta serie
        df_legislaturas_periodo = df_legislaturas[columna_nombre_periodo == periodo_nombre].copy()

        # 2. Crear carpeta de salida
        nombre_carpeta = sanitize_filename(periodo_nombre)
        carpeta_salida = DATA_DIR_RAW / nombre_carpeta
        carpeta_salida.mkdir(parents=True, exist_ok=True)

        # 3. Definir y guardar archivo de salida
        ruta_salida = carpeta_salida / "legislaturas.csv"
        
        # --- INICIO DE CORRECCIÓN (ORDENAMIENTO) ---
        # 4. Ordenar para consistencia.
        #    Sabemos (por el CSV) que la 'FechaInicio' de la *Legislatura*
        #    (no del período) está en la POSICIÓN 9 (índice 9).
        #    Para ordenar por una columna ambigua, obtenemos sus valores 
        #    por .iloc, obtenemos el índice de orden (.argsort),
        #    y re-ordenamos el DataFrame.
        
        try:
            # Obtener los valores de la columna de fecha de la legislatura (índice 9)
            sort_key_series = df_legislaturas_periodo.iloc[:, 9]
            
            # Obtener los índices que ordenarían esta serie
            sorted_indices = sort_key_series.argsort()
            
            # Reordenar el DataFrame usando los índices
            df_legislaturas_periodo_sorted = df_legislaturas_periodo.iloc[sorted_indices]
            
        except IndexError:
            logging.warning(f"No se pudo ordenar por la columna 9 para {periodo_nombre}. Guardando sin ordenar.")
            df_legislaturas_periodo_sorted = df_legislaturas_periodo
        # --- FIN DE CORRECCIÓN (ORDENAMIENTO) ---
        
        # 5. Guardar
        df_legislaturas_periodo_sorted.to_csv(ruta_salida, index=False, encoding="utf-8")
        
    except Exception as e:
        logging.error(f"✗ ERROR al procesar período {periodo_nombre}: {e}")

logging.info("--- Extracción de legislaturas finalizada ---")